In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
#hide
from fastbook import *
from fastai.vision.widgets import *
import requests


     |████████████████████████████████| 727kB 6.8MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 1.2MB 13.4MB/s 
     |████████████████████████████████| 194kB 31.4MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
Mounted at /content/gdrive


In [ ]:
def search_images_bing_new(key, term, customConfigId, min_sz=128):
	url = 'https://api.bing.microsoft.com/v7.0/custom/images/search?' + 'q=' + term + '&' + 'customconfig=' + customConfigId + '&' + 'count=150'
	r = requests.get(url, headers={'Ocp-Apim-Subscription-Key': key})
	search_results = r.json()
	return L([img["thumbnailUrl"] + ".jpg" for img in search_results["value"][:150]])


In [ ]:
hotdogs = search_images_bing_new('1be3ada0f2a649d087e9fb39798485a0', 'hotdog', 'efb1b149-c219-4091-be6d-14a279c405c4')

In [ ]:
hotdogs

(#150) ['https://tse2.mm.bing.net/th?id=OIP.WwNG0Db_o58XuR8uyHFGJQHaE8&pid=Api.jpg','https://tse3.mm.bing.net/th?id=OIP.1Xkr3YbMeVXI-QJL8w6AigHaDt&pid=Api.jpg','https://tse2.mm.bing.net/th?id=OIP.QHEOpw8npesuG0h6ojJ0HwHaFj&pid=Api.jpg','https://tse3.mm.bing.net/th?id=OIP.UyKMCHBsnEpwRGlJmu1HWQHaE8&pid=Api.jpg','https://tse2.mm.bing.net/th?id=OIP.3IVuG7q7Q3B_BmyXkvjz0gHaFj&pid=Api.jpg','https://tse4.mm.bing.net/th?id=OIP.uovloOsMkI8LfzTheiLThQHaE6&pid=Api.jpg','https://tse4.mm.bing.net/th?id=OIP.au202vJwBfwntBr4Byjw5wHaFj&pid=Api.jpg','https://tse1.mm.bing.net/th?id=OIP.nE5Gj2cAgzraJHOhzBG65AHaEK&pid=Api.jpg','https://tse3.mm.bing.net/th?id=OIP.WqTuusD_RbVuMEIbIDlFsQHaD4&pid=Api.jpg','https://tse1.mm.bing.net/th?id=OIP.P8GRsSEg7Eh6btc5vTDvCAHaEK&pid=Api.jpg'...]

In [ ]:
hotdogs[0]

'https://tse2.mm.bing.net/th?id=OIP.WwNG0Db_o58XuR8uyHFGJQHaE8&pid=Api.jpg'

In [ ]:
food_types = 'hotdog','burger'
path = Path('foods')
if not path.exists():
  path.mkdir()
  for f in food_types:
    dest = (path/f)
    dest.mkdir(exist_ok=True)
    results = search_images_bing_new('1be3ada0f2a649d087e9fb39798485a0', f, 'efb1b149-c219-4091-be6d-14a279c405c4')
    download_images(dest, urls=results)

In [ ]:
fns = get_image_files(path)
fns

(#300) [Path('foods/hotdog/00000059.jpg'),Path('foods/hotdog/00000034.jpg'),Path('foods/hotdog/00000015.jpg'),Path('foods/hotdog/00000002.jpg'),Path('foods/hotdog/00000095.jpg'),Path('foods/hotdog/00000025.jpg'),Path('foods/hotdog/00000096.jpg'),Path('foods/hotdog/00000019.jpg'),Path('foods/hotdog/00000044.jpg'),Path('foods/hotdog/00000023.jpg')...]

In [ ]:
failed = verify_images(fns)
failed

(#0) []

In [ ]:
class DataLoaders(GetAttr):
  def __init__(self, *loaders): self.loaders = loaders
  def __getitem__(self, i): return self.loaders[i]
  train, valid = add_props(lambda i, self: self[i])

In [ ]:
foods = DataBlock(
blocks=(ImageBlock, CategoryBlock),
get_items=get_image_files,
splitter=RandomSplitter(valid_pct=0.2, seed=42),
get_y=parent_label,
item_tfms=Resize(128))